<a href="https://colab.research.google.com/github/eughappy/HomeWork/blob/CNN/hw13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 part

In [21]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.utils import to_categorical

In [29]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape((60000, 28, 28, 1))
x_train= x_train.astype("float32") / 255
x_test = x_test.reshape((10000, 28, 28, 1))
x_test = x_test.astype("float32") / 255
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [7]:
model1 = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [8]:
model1.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])

In [9]:
history = model1.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 70s 74ms/step - loss: 0.6352 - accuracy: 0.7674 - val_loss: 0.3949 - val_accuracy: 0.8569
Epoch 2/5
938/938 [==============================] - 64s 69ms/step - loss: 0.3898 - accuracy: 0.8596 - val_loss: 0.3367 - val_accuracy: 0.8789
Epoch 3/5
938/938 [==============================] - 65s 70ms/step - loss: 0.3363 - accuracy: 0.8802 - val_loss: 0.3026 - val_accuracy: 0.8907
Epoch 4/5
938/938 [==============================] - 67s 71ms/step - loss: 0.3101 - accuracy: 0.8893 - val_loss: 0.2882 - val_accuracy: 0.8985
Epoch 5/5
938/938 [==============================] - 66s 71ms/step - loss: 0.2940 - accuracy: 0.8975 - val_loss: 0.2854 - val_accuracy: 0.9011


In [10]:
model2 = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    BatchNormalization(),
    Flatten(),

    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),

    Dense(10, activation='softmax')
])

In [11]:
model2.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
history2 = model2.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=5, batch_size=64)

Epoch 1/5
938/938 [==============================] - 86s 90ms/step - loss: 0.5225 - accuracy: 0.8171 - val_loss: 0.3486 - val_accuracy: 0.8692
Epoch 2/5
938/938 [==============================] - 86s 92ms/step - loss: 0.3485 - accuracy: 0.8765 - val_loss: 0.4092 - val_accuracy: 0.8545
Epoch 3/5
938/938 [==============================] - 86s 92ms/step - loss: 0.3075 - accuracy: 0.8905 - val_loss: 0.3036 - val_accuracy: 0.8891
Epoch 4/5
938/938 [==============================] - 84s 89ms/step - loss: 0.2816 - accuracy: 0.9007 - val_loss: 0.3098 - val_accuracy: 0.8885
Epoch 5/5
938/938 [==============================] - 84s 89ms/step - loss: 0.2652 - accuracy: 0.9051 - val_loss: 0.2827 - val_accuracy: 0.8991


Найкращою виявилась модель у якій застосовано BatchNormalization окрім дропаутів.

Порівняння результатів багатошарової нейронної мережі (MLP) та згорткової нейронної мережі (CNN):

Результати багатошарової нейронної мережі після навчання: 82.31%
Втрати після навчання: 0.4009


Результати згорткової нейронної мережі (CNN) після навчання: 90.51%
Втрати після навчання: 0.2652

Згорткова нейронна мережа  показала вищу точність на тестових даних (90.51%) у порівнянні з багатошаровою нейронною мережею  (82.31%). Проте навчається така модель довше: 7 хвилин, коли багатошарова нейронна мережа навчилася лише за 2-3 хвилини.

# PART 2

In [36]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [37]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [38]:
x_train = np.repeat(x_train, 3, axis=3)
x_test = np.repeat(x_test, 3, axis=3)

In [39]:
x_train = tf.image.resize(x_train, [32, 32])
x_test = tf.image.resize(x_test, [32, 32])

In [40]:
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
vgg16_base.trainable = False

In [41]:
model = Sequential([
    vgg16_base,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [42]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.0001)

In [43]:
history = model.fit(x_train, y_train, epochs=10, batch_size=256, validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Epoch 1/10
188/188 [==============================] - 791s 4s/step - loss: 2.3773 - accuracy: 0.6733 - val_loss: 0.6143 - val_accuracy: 0.7923 - lr: 0.0010
Epoch 2/10
188/188 [==============================] - 780s 4s/step - loss: 0.7052 - accuracy: 0.7622 - val_loss: 0.5336 - val_accuracy: 0.8152 - lr: 0.0010
Epoch 3/10
188/188 [==============================] - 719s 4s/step - loss: 0.5919 - accuracy: 0.7945 - val_loss: 0.4987 - val_accuracy: 0.8268 - lr: 0.0010
Epoch 4/10
188/188 [==============================] - 779s 4s/step - loss: 0.5321 - accuracy: 0.8084 - val_loss: 0.4775 - val_accuracy: 0.8319 - lr: 0.0010
Epoch 5/10
188/188 [==============================] - 780s 4s/step - loss: 0.4934 - accuracy: 0.8219 - val_loss: 0.4599 - val_accuracy: 0.8378 - lr: 0.0010
Epoch 6/10
188/188 [==============================] - 727s 4s/step - loss: 0.4705 - accuracy: 0.8294 - val_loss: 0.4491 - val_accuracy: 0.8441 - lr: 0.0010
Epoch 7/10
188/188 [==============================] - 783s 4s/st

In [44]:
for layer in vgg16_base.layers[-4:]:
    layer.trainable = True

In [45]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
history_fine = model.fit(x_train, y_train, epochs=2, batch_size=512,
                         validation_split=0.2, callbacks=[early_stopping, reduce_lr])

Epoch 1/2
94/94 [==============================] - 728s 8s/step - loss: 0.3665 - accuracy: 0.8632 - val_loss: 0.4220 - val_accuracy: 0.8560 - lr: 1.0000e-04
Epoch 2/2
94/94 [==============================] - 730s 8s/step - loss: 0.3568 - accuracy: 0.8670 - val_loss: 0.4194 - val_accuracy: 0.8577 - lr: 1.0000e-04


Порівняння результатів багатошарової нейронної мережі (MLP) та згорткової нейронної мережі (CNN на VGG16):

Результати багатошарової нейронної мережі після навчання: 82.31%
Втрати після навчання: 0.4009


Результати згорткової нейронної мережі (CNN на VGG16) після навчання та донавчання: 85.77%
Втрати після навчання: 0.3568

Згорткова нейронна мережа  показала вищу точність на тестових даних (85.77%) у порівнянні з багатошаровою нейронною мережею  (82.31%).
Згорткова модель на VGG16 навчаэтсья у рази довше, близько години,а результати не дуже кращі. Також ця згорткова нейронна мережа споживає більше пам'яті, ніж багатошарова нейронна мережа, через свою складну архітектуру.